# Mount Google drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Copy data zip files to Colab working space

In [ ]:
# 다운로드 받고 구글 드라이브에 해당 파일을 넣은 경로를 아래 cp -r '   '에  입력해주시면 됩니다.
!cp -r '/content/drive/My Drive/Copy of fish_data_small.zip' fish_data_small.zip
!cp -r '/content/drive/My Drive/Copy of fish_data.zip' fish_data.zip

# Extract zip files

In [ ]:
!unzip /content/fish_data_small.zip
!unzip /content/fish_data.zip

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import optimizers
%load_ext tensorboard
import os
import cv2
import numpy as np
import glob
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras import backend as K

In [ ]:
# // train_data path
train_path = "/content/fish_data_small/train/"
#// test_data path
test_path="/content/fish_data_small/test/"

# Get the data labels

In [ ]:
labels=[]
label_paths = glob.glob(train_path+"*")
for t_path in label_paths:
    labels.append(t_path[len(train_path):])
labels.sort()

#라벨들을 프린트 하시오
#code here

# Load training data

In [ ]:
picture_size = 150
train_data = []
train_data_label =[]

for label in labels:
    print (label)
    img_files = glob.glob(train_path+label+"/*.*")
    num_label = labels.index(label)
    print("data size for " + label + " : " + str(len(img_files)))
    for img_file in img_files:
        #print("loading "+img_file)
        img = cv2.imread(img_file)
        img = cv2.resize(img,(picture_size,picture_size))
        train_data.append(img)
        train_data_label.append(num_label)

In [ ]:
plt.imshow(train_data[1000])
print("Label: ", labels[train_data_label[1000]])
plt.show()

In [ ]:
len(train_data)

In [ ]:
train_data[0].shape

# Load test data

In [ ]:
picture_size = 150
test_data = []
test_data_label =[]

#test_data와 test_data_label에 적절한 데이터를 채우시오 (train_data 참고)
#code here



In [ ]:
len(test_data)

In [ ]:
test_data[0].shape

# Data preparation

In [ ]:
x = np.array(train_data)
y = np.array(train_data_label)

#train_test_split를 이용하여 훈련데이터셋과 검증데이터셋으로 나누기. 검증데이터셋 20%, 인자로 random_state = 230 지정
X_train, X_val, y_train, y_val = 

Y_train = to_categorical(y_train, len(labels))
Y_val = to_categorical(y_val, len(labels))

#test_data[]와 test_data_lable[]을 이용하여 테스트셋 생성
X_test =     # code here:  test_data 이용
y_test =     # code here: test_data_label 이용
Y_test =      # code here: y_test를 one_hot vector로 변환

# Data Augmentation

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale=1./255.,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)
# for training data, we apply normalize, randomly rotate, shift and flip, etc.

val_test_datagen = ImageDataGenerator(rescale=1./255.)
# for validation and test sets, we only normalize

train_generator = train_datagen.flow(X_train,Y_train,batch_size=64)
validation_generator = val_test_datagen.flow(X_val,Y_val,batch_size=64)
test_generator = val_test_datagen.flow(X_test,Y_test,batch_size=64)

# Build model

In [ ]:
from tensorflow.keras.applications import VGG16
conv_base = VGG16(weights='imagenet',
                  include_top=False,
                  input_shape=(150, 150, 3))
# by setting weights='imagenet', we load pre-trained weights (trained with ImageNet data) into VGG16 model.
# by setting include_top=False, we only use part of VGG16 architecture up to the last pooling layer

conv_base.trainable = False  # Freeze weights of conv_base layers (VGG16)

#Sequential()을 이용하여 model 초기화
model = #code here

#model에 베이스모델 추가
#code here

#model 에 Flatten층 추가
#code here

#model에 dense 층 추가, 뉴런 256개, 활성화함수 relu
#code here

#model에 dense 층 추가, 뉴런 10개, 활성화함수 softmax



In [ ]:
conv_base.summary()

In [ ]:
model.summary()

# Train model

In [ ]:
## 추가적으로 학습된 모델 가중치를 저장하는데 내가 원하는 경로에 저장하고 싶다면
import os
# exist_ok 인자에 True를 입력해주면 해당 폴더가 생성되어 있으면 이미 폴더가 생성된거라 에러가 발생해야 하는데 해당 에러를 무시해 준다.
# 그래서 저는 mkdir() 함수보단 makedirs() 함수를 자주 사용합니다.
os.makedirs("/content/drive/MyDriver/작업 폴더/Model_File/",exist_ok=True)

In [ ]:

#model compile 하기,optimizer는 RMSprop, 학습률 1e-3, 손실함수 categorical_crossentropy, metric acc
#code here

checkpoint_cb = keras.callbacks.ModelCheckpoint(
    "/content/drive/MyDriver/작업 폴더/Model_File/my_fish_model.h5",
    monitor='val_acc', verbose=1, save_best_only=True, mode='auto')

epoch_num = 20
#모델 훈련 시 콜백함수 등록하여 호출
history = model.fit(train_generator,
      epochs=epoch_num,
      validation_data= validation_generator,
      #code here 
      )



In [ ]:
#훈련시 저장된 최상의 모델 복원
new_model = #code here



In [ ]:
new_model.evaluate(test_generator)